# Hock-Schittkowsky 71 - Degenerate
(Adapted from Problem number [71](https://www.coin-or.org/Ipopt/documentation/node20.html) from the Hock-Schittkowsky test suite and augmented with the formulation 4.2 of Curtis, et al. SIAM Journal on Optimization. 2009 Sep 16;20(3):1224-49.)

> **Requirements**
> - python3.X
> - pyomo 5.X.X
> - [ipopt](https://github.com/coin-or/Ipopt) 3.12.12 (tested with)

### 0. Introduction

In this problem, an ill-posed problem is tested with *ipopt*; to demonstrate the effects of "bad" modelling.
Firstly, consider the problem:

\begin{align}
\text{minimize}_{x} \quad & x_{1}x_{4} \left(x_{1} + x_{2} + x_{3} \right) + x_{3} \\
\text{subject to} \quad & x_{1}x_{2}x_{3}x_{4} \geq 25, \\
& x_{1}^{2} + x_{2}^{2} + x_{3}^{2} + x_{4}^{2} = 40 \\
& 1 \leq x_{1}, x_{2}, x_{3}, x_{4} \leq 5,
\end{align}

With initial guess $x_{IG} = \left(1,5,5,1 \right)$. This problem is well-posed in the sense that the gradients of the constraints at the initial guess and at the solution $x^{*} = \left(1, 4.7443, 3.8211, 1.3794\right)$ are linearly independent.  
However, one can make a modelling mistakes and create problematic cases; particularly for solvers like *ipopt*.
For example, suppose that a constraint is added to the previous model so that:

\begin{align}
c_{1} \left(x\right) &= 0 ,\\
c_{1} \left(x \right) - c_{1}^{2} \left( x \right) & = 0,
\end{align}

where the last constraint contains information of the already defined $c_{1} \left(x \right)= x_{1}^{2} + x_{2}^{2} + x_{3}^{2} + x_{4}^{2} - 40 = 0$. Clearly, this constraint does not change the feasible set of the original problem. However, the linearized functions of these constraints are inconsistent at all points for which $c_{1} \neq 0$.

The purpose of this exercise is to notice the behaviour of the solver as a result of bad modelling.  
Although `AbstractModels` are used, note that they are not necessary to construct this problem.

### 1. Package imports and model declaration

For this model, the use of an *AbstractModel* will be shown. This kind of model is typical for problems whose structure depend on data. The *AbstractModel* will act as a blueprint for the overall model but it can *only* behave as a regular model after its instantiation. Note that for this case study a *ConcreteModel* would work just as fine.

In [3]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
m = AbstractModel()  #: Abstract model for educational purposes

### 2. Indexing sets and Variables

The problem HK071 has four variables $x_{1}, x_{2}, x_{3}, x_{4}$. It is also possible to create a vector $x \in \mathbb{R}^{4}$. 
In abstract models, the sets and parameter data can be passed from a [text file](https://ampl.com/BOOK/CHAPTERS/12-data.pdf), the data can be also be written in tabular form.  
It is known that the set of variables goes from 1...4, however this information can be part of the data file of the abstract model.

In [4]:
m.i = Set()  #: No particular initialization here.

In [5]:
x_guess = {1: 1, 2: 5, 3: 5, 4:1}

The variables' initial value can be declared within the data file as well. Since the bounds of the variables are the same, it is possible to use the `bounds` keyword to assign the same bounds for all the variables.

In [6]:
#m.x = Var(m.i, initialize=x_guess, bounds=(1,5))
m.x = Var(m.i, bounds=(1,5))

### 3. Constraints and Objective

In [7]:
def _c0_rule(mod):
    return mod.x[1] * mod.x[2] * mod.x[3] * mod.x[4] >= 25
m.c0 = Constraint(rule=_c0_rule)  #: nothing special about this one

As noted in other examples meaningful *Constraint* objects require an expression. Moreover, there exists *Expression* objects that encapsulate symbolic relationships between variables. *Expression* objects can be conviniently used, for example, to initialize parts of a *Constraint* object. In the following example an *Expression* object is used to set the right-hand-side of the second constraint of the model.

In [8]:
m.e1 = Expression(rule=lambda mod: mod.x[1]**2 + mod.x[2]**2 + mod.x[3]**2 + mod.x[4]**2 - 40)  #: the expression is declared

Then the expression is passed to the constraint objects.

In [9]:
m.c1 = Constraint(expr=m.e1 == 0)
m.c2 = Constraint(expr=m.e1 - m.e1 ** 2 == 0)

Finally, the objective is also defined using a python function:

In [10]:
def _objective_rule(mod):
    return mod.x[1] * mod.x[1] * (mod.x[1] + mod.x[2] + mod.x[3]) + mod.x[3]
m.of = Objective(rule=_objective_rule, sense=minimize)

Note that for the *AbstractModel* class, the constraints require python functions to pass on the respective expressions using the `rule` keyword.

### 4. Solution and discussion

In [11]:
with open("hk071.dat", "w") as f:
    f.write("set i := 1 2 3 4;\n")
    f.close()

Before solving the problem an instance of the model must be constructed. If there is a data file, the construction of the instance can be done using the `create_instance` method as follows,

In [12]:
instance = m.create_instance("hk071.dat")

At this point the instance of the model has been created. The instance basically behaves like a `ConcreteModel` and can be modified if necessary. For example, the variables can be set to the initial guess using the `set_value` method

In [13]:
instance.x[1].set_value(1)
instance.x[2].set_value(5)
instance.x[3].set_value(5)
instance.x[4].set_value(1)

Then the solver can be created and called to solve the problem.

In [14]:
opt = SolverFactory('ipopt')
try:
    res = opt.solve(instance, tee=True)
except ValueError:
    print("Warning, something went wrong!")

/tmp/tmpbcxxs81q.pyomo.nl -AMPL 
Ipopt vanilla0: 
This is Ipopt version vanilla0, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        8
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.6109693e+01 1.15e+02 2.31

 214r 7.3651359e+00 3.04e-04 1.00e+03  -3.5 0.00e+00  19.5 0.00e+00 0.00e+00R  1 1.17e-04 0.00e+00 1.00e+03 
 215r 7.8075829e+00 5.07e-02 6.29e+02  -3.5 7.79e+01    -  9.90e-01 8.28e-04f  1 9.83e-05 5.35e-01 1.00e+01 
 216r 7.8138436e+00 1.28e-02 1.23e+02  -3.5 2.83e-03    -  6.82e-01 7.48e-01f  1 3.41e-05 8.73e+02 3.18e+00 Nhj 
 217r 7.8157815e+00 2.58e-03 4.34e+01  -3.5 4.71e-03    -  9.90e-01 7.92e-01h  1 4.98e-06 6.95e+02 8.26e-02 
 218r 7.8259326e+00 2.98e-05 1.11e+00  -3.5 9.57e-03    -  9.81e-01 1.00e+00h  1 1.03e-06 5.94e+02 1.38e-02 
 219r 7.8367141e+00 2.07e-04 3.20e-01  -3.5 3.09e-02   0.0 1.00e+00 1.00e+00h  1 1.16e-06 3.02e+02 3.04e-02 
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls  spn     lmbd_max z_max
 220r 7.9446725e+00 7.26e-04 5.24e+00  -3.5 1.76e-02    -  9.84e-01 1.00e+00h  1 1.21e-06 1.83e+01 5.41e-03 
 221r 8.3444750e+00 2.75e-03 2.58e+01  -3.5 1.00e+00    -  9.14e-01 1.00e+00h  1 1.22e-06 3.13e+02 6.99e-03 
 222r 8.3438094e+00

If *MUMPS* is used as linear solver and the default options are used, the solution will fail. In this case, the default output of ipopt reveals that the column *inf_du* has grown significantly right before termination. It is likely related to the failure to satisfy the *constraint qualifications* given the current 
formulation of the problem.  
It is clear that a slight mistake in the modelling can be problematic. Nevertheless, for this case trying a different starting point or different options *might* actually help solving the problem.

### Optional 1: Fix the model

The model can be easily fixed if the last constraint is removed. This can be done by using the `deactivate` method.

In [15]:
instance.c2.deactivate()

And then solve again.

In [16]:
res = opt.solve(instance, tee=True)

/tmp/tmpgi5yafob.pyomo.nl -AMPL 
Ipopt vanilla0: 
This is Ipopt version vanilla0, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  8.3234331e+00 9.61e-12 5.34

In [17]:
instance.x.pprint()

x : Size=4, Index=i
    Key : Lower : Value             : Upper : Fixed : Stale : Domain
      1 :     1 :               1.0 :     5 : False : False :  Reals
      2 :     1 :  3.44948966996681 :     5 : False : False :  Reals
      3 :     1 : 1.449489760694747 :     5 : False : False :  Reals
      4 :     1 :               5.0 :     5 : False : False :  Reals


### Optional 2: examine the ipopt output
Often, the output contains extremely important information. Although, it might be difficult to interpret; a large difference between the scales of the *inf_du* and *inf_pr* might signal failure of constraint qualifications.
Also, one might try different options in ipopt.  
For instance:

In [18]:
with open("ipopt.opt", "w") as f:
    f.write("print_info_string yes\n")

Try again with the bad constraint. Now, sometimes there is a new column at the end of the ipopt output.

In [19]:
instance.c2.activate()
opt.solve(instance, tee=True)

/tmp/tmpmr88347b.pyomo.nl -AMPL 
Ipopt vanilla0: 
This is Ipopt version vanilla0, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        8
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  7.4772395e+00 5.21e-01 1.30

  70r 8.1311504e+00 7.53e-05 6.19e+02  -5.7 6.55e-03    -  1.00e+00 9.77e-04h 11 7.49e-09 3.75e+01 3.80e-05 F+
  71r 8.1345395e+00 2.30e-07 6.02e-01  -5.7 6.54e-03    -  1.00e+00 1.00e+00h  1 1.10e-08 4.47e+02 3.80e-05 
  72r 8.1345471e+00 8.24e-13 1.09e-03  -5.7 1.47e-05    -  1.00e+00 1.00e+00h  1 6.00e-09 2.04e+02 3.80e-05 
  73  8.1345450e+00 9.31e-13 1.38e+01  -3.8 1.42e+00    -  3.12e-01 1.87e-07f 19 S
  74  8.1345440e+00 1.48e-12 1.38e+01  -3.8 3.91e+00    -  3.71e-01 1.52e-07f 19 
  75  8.1345440e+00 1.48e-12 1.38e+01  -3.8 3.65e-19  19.6 1.00e+00 1.00e+00   0 L
  76  8.1345440e+00 1.48e-12 1.38e+01  -5.7 1.09e-18  19.1 1.00e+00 1.00e+00T  0 L
  77  7.9193145e+00 3.84e-02 1.29e+08  -8.6 1.90e+02    -  2.08e-02 8.81e-03f  1 
  78r 7.9193145e+00 3.84e-02 9.99e+02  -1.4 0.00e+00  18.1 0.00e+00 4.77e-07R 22 5.01e-02 0.00e+00 5.02e-05 
  79r 1.2849069e+01 1.64e-01 2.21e+00  -1.4 1.04e+04    -  1.00e+00 9.93e-04f  1 1.78e-02 2.20e-03 2.22e+00 
iter    objective    inf_pr   inf_du lg(

Total CPU secs in NLP function evaluations           =      0.005

EXIT: Solved To Acceptable Level.


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 4, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt vanilla0\\x3a Solved To Acceptable Level.', 'Termination condition': 'optimal', 'Id': 1, 'Error rc': 0, 'Time': 0.13631010055541992}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Typically, the `L`, `q` and `s` are not good signs (https://www.coin-or.org/Ipopt/documentation/node37.html)

### Credits:

- David Thierry (Carnegie Mellon University @2019)